# HW3

## Text Processing

### Q1

1. Modify the code I wrote in lecture 8 with what you have learnt in lecture 9 and correctly tokenize the text both on the word and sentence level, and by removing the stopwords. Rewrite the `getSummary` function and all the other functions that it depends by maing these corrections.

2. Rewrite the code I wrote for `getKeywords` function making the same corrections.

3. Test your code from parts 1 and 2 on random articles from the Guardian.

4. Rewrite the `getSubjectGuardian` function for another newspaper in English, and test your code from part 1 and 2 on random articles from this new newspaper.

<hr>

NLTK large scale'de yavaş olduğundan Fast Tokenizer kullanabiliriz.

In [64]:
import requests
import nltk
import regex as re
import numpy as np

In [90]:
from xmltodict import parse

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer

In [165]:
def processText(text):
    raw_sentences = sent_tokenize(text)
    raw_words = word_tokenize(text)
    
    res_text = {'sentences': raw_sentences,
                    'words': raw_words}
    
    res_text.update({'cleanedSentences': [re.sub(r'[^\p{Letter}\s]','',sentence.lower()) for sentence in res_text['sentences']]})
    
    tmp = [re.sub(r'[^\p{Letter}]','',word.lower()) for word in res_text['words']]
    words_nonempty = list(filter(lambda x: x != '', tmp))
    res_text.update({'cleanedWords': words_nonempty})
    

    return res_text

In [176]:
text = 'My name is Mr. Smith. I have a Ph.D. from M.I.T. and I work at I.B.M. Now, look at pg. 12 of your text.'
s = processText(text)['cleanedSentences']
w = processText(text)['cleanedWords']
(s,w)

(['my name is mr smith',
  'i have a phd from mit',
  'and i work at ibm',
  'now look at pg',
  ' of your text'],
 ['my',
  'name',
  'is',
  'mr',
  'smith',
  'i',
  'have',
  'a',
  'phd',
  'from',
  'mit',
  'and',
  'i',
  'work',
  'at',
  'ibm',
  'now',
  'look',
  'at',
  'pg',
  'of',
  'your',
  'text'])

In [153]:
def getMatrix(sentences):
    vectorizer = CountVectorizer()
    return vectorizer.fit_transform(sentences)

In [177]:
def getSummary(text,k):
    
    sentences = processText(text)['cleanedSentences']
    
    matrix = getMatrix(sentences)
    projection = PCA(n_components=1)
    weights = projection.fit_transform(matrix.toarray())
    res = list(zip(weights.transpose()[0],range(112),sentences))
    tmp = sorted(res,key=lambda x: x[0],reverse=True)[:k]
    return sorted(tmp, key=lambda x: x[1])

In [179]:
def getKeywords(text,sw,k):
    sentences = processText(text)['cleanedSentences']
    
    words = processText(text)['cleanedWords']
    words_notstop = [w for w in words if not w in sw]

    vectorizer = CountVectorizer(stop_words=sw)
    matrix = vectorizer.fit_transform(sentences)
    
    projection = PCA(n_components=1)
    tmp = projection.fit_transform(matrix.transpose().toarray())
    weights = tmp.transpose()[0]
    
    return sorted(zip(weights,words_notstop),key=lambda x: x[0], reverse=True)[:k]

I will use this function for pulling the RSS feeds of The Guardian

In [262]:
def getSubjectGuardian(subject):
    with requests.get(f'https://www.theguardian.com/{subject}/rss') as link:
        raw = parse(link.text)
    return raw['rss']['channel']['item']

I overwrite the function for the RSS feeds of The Sun Daily.

In [263]:
def getSubjectGuardian2(subject):
    with requests.get(f'https://www.thesundaily.my/rss/{subject}.xml') as link:
        raw = parse(link.text)
    return raw['rss']['channel']['item']

In [261]:
from bs4 import BeautifulSoup

def getText(url):
    with requests.get(url) as link:
        tags = BeautifulSoup(link.content,'html.parser')
        
    return ' '.join([x.text for x in tags.find_all('p')])

In [293]:
articles_theGuardian = getSubjectGuardian('sport')
n = len(articles_theGuardian)

articles_theSun = getSubjectGuardian2('Sport')
m = len(articles_theSun)

k = np.random.randint(n)
l = np.random.randint(m)

article_theGuardian = getText(articles_theGuardian[k]['link'])
articletitle_theGuardian = articles_theGuardian[k]['title']

article_theSun = getText(articles_theSun[l]['link'])
articletitle_theSun = articles_theSun[l]['title']

(articletitle_theGuardian,articletitle_theSun)

("'We deserve the spotlight': Taylor and Serrano face off before iconic fight – video",
 'Benfica reach agreement in principle with Schmidt to become new boss')

In [294]:
getSummary(article_theGuardian,3)

[(-0.10074684842444077,
  0,
  'boxers katie taylor and amanda serrano faced off on top of the empire state building to launch an historic fight week'),
 (4.2520232478512785,
  2,
  'this fight is already making history as the biggest female fight of all time as it looks to put female boxing more firmly on the map'),
 (-1.5302682661951896,
  3,
  'source ap   reuters \n\n        \n            wed  apr  \xa0bst\n\n\n            last modified on wed  apr  \xa0bst')]

In [296]:
swEN = set(stopwords.words('english'))
getKeywords(article_theGuardian,swEN,15)

[(1.5453877337298734, 'building'),
 (1.5453877337298734, 'serrano'),
 (0.7440573328906237, 'world'),
 (0.7440573328906237, 'champion'),
 (0.7440573328906237, 'head'),
 (0.7440573328906237, 'taylor'),
 (0.7440573328906237, 'night'),
 (0.7440573328906237, 'garden'),
 (0.7440573328906237, 'making'),
 (0.7440573328906237, 'biggest'),
 (0.7440573328906237, 'fight'),
 (0.7440573328906237, 'boxing'),
 (0.7440573328906237, 'source'),
 (0.6923167055213204, 'history'),
 (0.6923167055213204, 'looks')]

In [297]:
getSummary(article_theSun,3)

[(2.8036779880938503,
  1,
  'benfica said that negotiations were taking place for german schmidt whose contract at the dutch eredivisie club expires at the end of the season to replace nelson verissimo'),
 (-0.15589731794032596,
  2,
  'the club added that the deal was not yet formalised'),
 (3.693264678038084,
  4,
  'he then took over at dutch side psv who last month said that former manchester united and real madrid forward ruud van nistelrooy had signed a threeyear contract to become the head coach from next season')]

In [299]:
getKeywords(article_theSun,swEN,15)

[(0.769154357290961, 'taking'),
 (0.769154357290961, 'place'),
 (0.769154357290961, 'club'),
 (0.769154357290961, 'replace'),
 (0.769154357290961, 'verissimo'),
 (0.769154357290961, 'deal'),
 (0.769154357290961, 'name'),
 (0.769154357290961, 'salzburg'),
 (0.769154357290961, 'guoan'),
 (0.769154357290961, 'psv'),
 (0.769154357290961, 'real'),
 (0.769154357290961, 'madrid'),
 (0.769154357290961, 'threeyear'),
 (0.769154357290961, 'become'),
 (0.769154357290961, 'head')]

### Q2

Write a function that returns all named entities (proper names, country names, corporation names only) from a URL. Function should take the URL as the input and must return the list of named entities from that URL. Test your code on random articles from the Guardian. Don't use the NLTK's NER that I demonstrated during the lecture. Use the SpaCY's NER function.

I downloaded spacy with this code :
```import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en```

I have used spacy.explain(token.label_) for pulling Proper Names, Country Names and Corporation Names. Then, I decided to take the tokens if whose label_ are PERSON, GPE or ORG.

In [410]:
import spacy

def SpacyNER(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    res = [ (token.text,token.label_,spacy.explain(token.label_)) for token in doc.ents if token.label_ in ['GPE','PERSON','ORG']]
    
    #sentences = sent_tokenize(text)
    #res = []
    #for i in range(len(sentences)):
    #    doc = nlp(sentences[i])
    #   res.append([ (token.text,token.label_,spacy.explain(token.label_)) for token in doc.ents])


    return res

In [411]:
articles_theGuardian = getSubjectGuardian('sport')
n = len(articles_theGuardian)
k = np.random.randint(n)
article_theGuardian = getText(articles_theGuardian[k]['link'])

SpacyNER(article_theGuardian)

[('Leicester', 'PERSON', 'People, including fictional'),
 ('Gianluca Mancini', 'PERSON', 'People, including fictional'),
 ('this Europa Conference League',
  'ORG',
  'Companies, agencies, institutions, etc.'),
 ('Stadio Olimpico', 'PERSON', 'People, including fictional'),
 ('José Mourinho', 'PERSON', 'People, including fictional'),
 ('England', 'GPE', 'Countries, cities, states'),
 ('Leicester', 'ORG', 'Companies, agencies, institutions, etc.'),
 ('Roma', 'PERSON', 'People, including fictional'),
 ('Nicola Zalewski', 'PERSON', 'People, including fictional'),
 ('Lorenzo Pellegrini', 'PERSON', 'People, including fictional'),
 ('Kasper Schmeichel', 'PERSON', 'People, including fictional'),
 ('Tottenham', 'ORG', 'Companies, agencies, institutions, etc.'),
 ('Jamie Vardy', 'PERSON', 'People, including fictional'),
 ('Chris Smalling', 'PERSON', 'People, including fictional'),
 ('Serie', 'ORG', 'Companies, agencies, institutions, etc.'),
 ('Tammy Abraham', 'PERSON', 'People, including fictio

### Q3

1. Write a function that returns the most positive and the most negative sentences from a text. The function must take the text as the input and must return a 2-tuple: the first element as the most positive and the second as the most negative sentence with their polarity scores.

2. Test your function on random articles from the Guardian.

In [417]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\bjk_a\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [464]:
def Sentiments(text):
    analyzer = SentimentIntensityAnalyzer()
    analyzer.polarity_scores(text)
    sentences = sent_tokenize(text)
    res = [(x,analyzer.polarity_scores(x)) for x in sentences]
    
    # Getting list of Negative Sentences and Positive Sentences
    list_neg = [ (sentence[0] , sentence[1]['neg']) for sentence in res]
    list_pos = [ (sentence[0] , sentence[1]['pos']) for sentence in res]
    
    # Getting most negative and most positive
    max_neg = max(list_neg,key=lambda x:x[1])
    max_pos = max(list_pos,key=lambda x:x[1])
    return ({'most negative':max_neg},
            {'most positive':max_pos})

In [465]:
articles_theGuardian = getSubjectGuardian('sport')
n = len(articles_theGuardian)
k = np.random.randint(n)
article_theGuardian = getText(articles_theGuardian[k]['link'])

Sentiments(article_theGuardian)

({'most negative': ('After Saturday’s bout, Fury said he was not ruling out exhibition fights or a return to World Wrestling Entertainment.',
   0.264)},
 {'most positive': ('I’ve got success, fame, glory.', 0.835)})

We check whether it is true or false.

In [467]:
text = article_theGuardian
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(text)
sentences = sent_tokenize(text)
[(x,analyzer.polarity_scores(x)) for x in sentences]

[('Tyson Fury has reiterated that he is retiring from boxing, stressing that “no amount of money” will tempt him to change the decision.',
  {'neg': 0.228, 'neu': 0.772, 'pos': 0.0, 'compound': -0.7351}),
 ('The 33-year-old undefeated WBC heavyweight champion once again declared himself happy with his lot and ready to hang up his gloves, three days on from beating his fellow Briton Dillian Whyte via a sixth-round stoppage in front of a 94,000 crowd at Wembley.',
  {'neg': 0.06, 'neu': 0.739, 'pos': 0.202, 'compound': 0.7964}),
 ('Speaking on Piers Morgan Uncensored on TalkTV, Fury said: “This is the truth, the gospel truth, nothing but the truth - I’m done.',
  {'neg': 0.088, 'neu': 0.677, 'pos': 0.235, 'compound': 0.4404}),
 ('Every good dog has its day.',
  {'neg': 0.0, 'neu': 0.633, 'pos': 0.367, 'compound': 0.4404}),
 ('Like a great Roman leader said, there will always be somebody else to fight.',
  {'neg': 0.135, 'neu': 0.521, 'pos': 0.344, 'compound': 0.6124}),
 ('“When is enough